In [3]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier 
import lightgbm as lgb
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [5]:
# GET INDICIES OF REAL TEST DATA FOR FE
#######################
# TAKE FROM YAG320'S KERNEL
# https://www.kaggle.com/yag320/list-of-fake-samples-and-public-private-lb-split
df_test = pd.read_csv('test.csv')
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
#  np.zeros_like(df_test).shape = (200000, 200)
unique_count = np.zeros_like(df_test)
# df_test.shape[1] =200
for feature in tqdm_notebook(range(df_test.shape[1])):
    # _= unique값, index_ = unique 값의 첫 번째 등장하는 index, count_ = unique 값의 count
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    
    unique_count[index_[count_ == 1], feature] += 1

# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

print('Real test data',len(real_samples_indexes))
print('Fake test data',len(synthetic_samples_indexes))


Real test data 100000
Fake test data 100000


In [124]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_id = test['ID_code']
real_test = test.loc[real_samples_indexes]
real_test_id = real_test.ID_code

In [63]:
print('train set Shape :', train.shape)
print('test set Shape :',test.shape)
print('Real test set Shape :',real_test.shape)
del test

train set Shape : (200000, 202)
test set Shape : (200000, 201)
Real test set Shape : (100000, 201)


In [64]:
real_test['target'] = -1
comb = pd.concat([train,real_test],axis=0,sort=True)

In [134]:
#frequency encoding
for i in tqdm_notebook(range(200)):
    fe = comb['var_'+str(i)].value_counts()
    name = 'var_'+str(i)+'_FE'
    comb[name] = comb['var_'+str(i)].map(fe)
train = comb[:len(train)]
test = comb[len(train):]

In [135]:
print(train.shape)
print(test.shape)

(200000, 402)
(100000, 402)


In [136]:
target = train['target']
del train['target']
del test['target']

train.drop('ID_code',axis=1 , inplace=True)
test.drop('ID_code',axis=1 , inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [174]:
auc = np.zeros(len(train))
predictions = np.zeros(len(test))
num_vars = 200
for j in tqdm_notebook(range(num_vars)):
    
    # MODEL WITH MAGIC
    features = ['var_'+str(j),'var_'+str(j)+'_FE']
    
    #  Bayesian Optimization             
    model = lgb.LGBMClassifier(** {'bagging_fraction': 0.7693,
   'bagging_freq': 2,'lambda_l1': 0.7199,'lambda_l2': 1.992,'learning_rate': 0.009455,'max_depth': 3,'min_data_in_leaf': 22,
   'min_gain_to_split': 0.06549, 'min_sum_hessian_in_leaf': 18.55,'num_leaves': 20,'feature_fraction': 1,'save_binary': True,
   'objective': 'binary','boosting_type': 'gbdt','verbosity': -1, 'metric': 'auc','is_unbalance': True,'boost_from_average': 'false',
   'num_threads': 6})

    model = model.fit(train[features], target)
    auc += model.predict_proba(train[features])[:,1]
    predictions += model.predict_proba(test[features])[:,1]
    
    print("CV score: {:<8.5f}".format(roc_auc_score(target, auc)))

CV score: 0.55132 
CV score: 0.57816 
CV score: 0.60109 
CV score: 0.60391 
CV score: 0.60580 
CV score: 0.61538 
CV score: 0.63498 
CV score: 0.63589 
CV score: 0.63876 
CV score: 0.64950 
CV score: 0.65025 
CV score: 0.65342 
CV score: 0.66970 
CV score: 0.68065 
CV score: 0.68165 
CV score: 0.68336 
CV score: 0.68420 
CV score: 0.68480 
CV score: 0.69244 
CV score: 0.69406 
CV score: 0.69628 
CV score: 0.70639 
CV score: 0.71545 
CV score: 0.71738 
CV score: 0.71986 
CV score: 0.72066 
CV score: 0.73088 
CV score: 0.73132 
CV score: 0.73302 
CV score: 0.73348 
CV score: 0.73381 
CV score: 0.73534 
CV score: 0.73858 
CV score: 0.74422 
CV score: 0.74924 
CV score: 0.75252 
CV score: 0.75578 
CV score: 0.75639 
CV score: 0.75681 
CV score: 0.75720 
CV score: 0.76295 
CV score: 0.76334 
CV score: 0.76390 
CV score: 0.76554 
CV score: 0.77195 
CV score: 0.77356 
CV score: 0.77394 
CV score: 0.77437 
CV score: 0.77668 
CV score: 0.77939 
CV score: 0.78001 
CV score: 0.78219 
CV score: 0.

In [175]:
subreal = pd.DataFrame({"ID_code": real_test_id.values})
subreal['target']=predictions
sub = pd.DataFrame({"ID_code": test_id})

In [176]:
finalsub = sub.set_index('ID_code').join(subreal.set_index('ID_code')).reset_index()
finalsub.fillna(0,inplace=True)
finalsub.to_csv('top1% submission.csv', index=False)